In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import re

from gensim.models import word2vec
from sklearn.base import TransformerMixin
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.tree import ExtraTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, \
    mean_squared_error
from sklearn.pipeline import Pipeline

from tsfresh.feature_extraction import extract_features

from xgboost import XGBRegressor
import optuna

In [8]:
MODELS = {'ExtraTreeRegressor': ExtraTreeRegressor,
          'Ridge': Ridge
          }

COL_TO_DEL = ['patient_id', 'patient_id_sum_values', 'patient_id_mean',
              'patient_id_maximum', 'patient_id_minimum',
              'patient_id_standard_deviation', 'patient_id_absolute_maximum',
              'visit_month_sum_values', 'visit_month_mean',
              'visit_month_maximum', 'visit_month_minimum',
              'visit_month_standard_deviation', 'visit_month_absolute_maximum']

TARGETS = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']

In [9]:
SETTINGS_LEVEL_1 = {'sum_values': None,
                    'mean': None,
                    'maximum': None,
                    'minimum': None,
                    'standard_deviation': None,
                    'absolute_maximum':  None,
                    'autocorrelation': [{'lag': 0}, {'lag': 1},
                                        {'lag': 2}, {'lag': 3},
                                        {'lag': 4}, {'lag': 5},
                                        {'lag': 6}],
                    'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
                                            {'f_agg': 'median', 'maxlag': 40},
                                            {'f_agg': 'var', 'maxlag': 40}]
                    }

SETTINGS_LEVEL_2 = {'sum_values': {'mean': None},
                    'mean': {'mean': None},
                    'length': {'mean': None,
                               'maximum': None,
                               'minimum': None},
                    'maximum': {'maximum': None,
                                'mean': None},
                    'minimum': {'minimum': None,
                                'mean': None},
                    'standard_deviation': {'mean': None},
                    'autocorrelation': {'mean': None}}

SETTINGS_LEVEL_3 = {'sum_values': None,
                    'mean': None,
                    'maximum': None,
                    'minimum': None,
                    'standard_deviation': None,
                    'absolute_maximum':  None}

In [10]:
def _split_all_data(peps, prots, clinical, train_patients, val_patients):
    train_peps = peps[peps.patient_id.isin(train_patients)] \
        .reset_index(drop=True)
    val_peps = peps[peps.patient_id.isin(val_patients)] \
        .reset_index(drop=True)

    train_prots = prots[prots.patient_id.isin(train_patients)] \
        .reset_index(drop=True)
    val_prots = prots[prots.patient_id.isin(val_patients)] \
        .reset_index(drop=True)

    train_clinical = clinical[clinical.patient_id.isin(train_patients)] \
        .reset_index(drop=True)
    val_clinical = clinical[clinical.patient_id.isin(val_patients)] \
        .reset_index(drop=True)
    return train_peps, val_peps, train_prots, val_prots, \
        train_clinical, val_clinical


def split_train_val_data(peps, prots, clinical, test_size, random_state):
    train_patients, val_patients = train_test_split(peps.patient_id.unique(),
                                                    test_size=test_size,
                                                    random_state=random_state)

    train_peps, val_peps, train_prots, val_prots, \
        train_clinical, val_clinical = _split_all_data(peps, prots,
                                                       clinical,
                                                       train_patients,
                                                       val_patients)

    return train_peps, val_peps, train_prots, val_prots, \
        train_clinical, val_clinical


def split_cv_data(peps, prots, clinical, train_idxs, val_idxs):

    train_peps, val_peps, train_prots, val_prots, \
        train_clinical, val_clinical = _split_all_data(peps, prots,
                                                       clinical,
                                                       train_idxs,
                                                       val_idxs)

    return train_peps, val_peps, train_prots, val_prots, \
        train_clinical, val_clinical

In [11]:
def create_config_pep(columns):

    config_pep = {col: {} for col in columns}
    config_pep[columns[0]].update({'length': None})

    for col in columns:
        config_pep[col].update(SETTINGS_LEVEL_1)

    return config_pep


def create_config_pep_prot(columns):

    config_pep_prot = {col: {} for col in columns}
    config_pep_prot[columns[0]].update({'length': None})

    for col in columns:
        for key in SETTINGS_LEVEL_2:
            if key in col:
                config_pep_prot[col] \
                    .update(SETTINGS_LEVEL_2[key])

    return config_pep_prot


def create_config_prot(columns):

    config_prot = {col: {} for col in columns}
    config_prot[columns[0]].update({'length': None})

    for col in columns:
        stop_indicator = False
        for key in SETTINGS_LEVEL_2:
            # stats for aggregative features
            if key in col:
                config_prot[col].update(SETTINGS_LEVEL_2[key])
                stop_indicator = True
        # stats for non aggregative features
        if not stop_indicator:
            config_prot[col].update(SETTINGS_LEVEL_3)

    return config_prot

In [20]:
def smape_func(y, y_pred, **kwargs):
    return 100/len(y) * np.sum(2 * np.abs(y_pred - y) /
                               (np.abs(y) + np.abs(y_pred)))

In [21]:
class W2vPepWrapper(TransformerMixin):
    """A class for applying word2vec for 
    encoding each character of peptides
    """
    def __init__(self, vector_size=30, window=5):
        self.vector_size = vector_size
        self.window = window
        self.cols = ['vector_pep_value_{}'.format(i)
                     for i in range(vector_size)]

    def _split_sequence(self, sequence):

        # We will separate single letters and UniMod strings
        # For example we want to split string 'ABCD(Unimod_4)EF like that:
        # A, B, C, D, Unimod, 4, E, F
        list_letters = sequence.split(r'(UniMod_')
        list_unimods = re.findall(r'\(UniMod_', sequence)

        list_letters = [letters.replace(')', '') for letters in list_letters]
        list_letters = [list(letters) for letters in list_letters]

        list_unimods = [unimod.replace(r'(', '') for unimod in list_unimods]
        list_unimods = [unimod.replace('_', '') for unimod in list_unimods]

        splitted_sequence = list_letters[0]

        if len(list_letters) > 1:

            for num in range(len(list_letters) - 1):
                splitted_sequence.append(list_unimods[num])
                splitted_sequence += list_letters[num + 1]

        elif len(list_unimods):
            splitted_sequence += list_unimods[0]

        return splitted_sequence

    def _create_vectors_df(self, splitted_sequence):
        vector_values = [self.w2v_model.wv.get_vector(symb)
                         for symb in splitted_sequence]
        vector_values = np.array(vector_values)
        vectors_df = pd.DataFrame(vector_values,
                                  columns=self.cols)
        vectors_df.index.name = 'Position'
        return vectors_df

    def _transform_peptides_to_vecs(self, peps):
        splitted_peps = self._split_sequence(peps.values[0])
        vec_df = self._create_vectors_df(splitted_peps)
        return vec_df

    def fit(self, X, y):
        peps, _, _ = X
        unique_peps = peps.Peptide.unique().astype('str')
        splitted_peps = [self._split_sequence(peptide_name)
                         for peptide_name in unique_peps]
        self.w2v_model = word2vec.Word2Vec(splitted_peps,
                                           vector_size=self.vector_size,
                                           window=self.window,
                                           workers=4)
        return self

    def transform(self, X):

        peps, prots, order = X
        peps_vectorized = peps.groupby('Peptide').Peptide \
            .apply(lambda x: self._transform_peptides_to_vecs(x))
        peps_vectorized = peps_vectorized.reset_index()
        return (peps, prots, order, peps_vectorized)


class W2vProtWrapper(TransformerMixin):
    """A class for applying word2vec for 
    encoding the names of proteins
    """
    def __init__(self, vector_size=30, window=100):
        self.vector_size = vector_size
        self.window = window
        self.cols = ['vector_prot_value_{}'.format(i)
                     for i in range(vector_size)]

    def _create_vectors_df(self, protein):
        if protein in self.w2v_model.wv:
            vector_values = self.w2v_model.wv.get_vector(protein)
        else:
            vector_values = self.w2v_model.wv.get_mean_vector(protein)
        vector_values = np.array(vector_values).reshape(1, -1)
        vectors_df = pd.DataFrame(vector_values, columns=self.cols)
        return vectors_df

    def _transform_protein_to_vecs(self, protein):

        vec_df = self._create_vectors_df(protein.values[0])
        return vec_df

    def fit(self, X, y):
        _, _, _, agg_prots = X
        prot_combinations = agg_prots.groupby('visit_id')['UniProt'] \
            .apply(lambda x: x.values).values
        prot_combinations = [list(prot_combination)
                             for prot_combination in prot_combinations]
        self.w2v_model = word2vec.Word2Vec(prot_combinations,
                                           vector_size=self.vector_size,
                                           window=self.window,
                                           workers=4)
        return self

    def transform(self, X):
        peps, prots, order, agg_prots = X
        agg_prots_vectorized = agg_prots.copy()
        prot_vectors = agg_prots_vectorized.groupby(['visit_id', 'UniProt']) \
            .UniProt.apply(lambda x: self._transform_protein_to_vecs(x))
        prot_vectors = prot_vectors.reset_index()
        agg_prots_vectorized = agg_prots_vectorized \
            .merge(prot_vectors, on=['visit_id', 'UniProt'])
        agg_prots_vectorized = agg_prots_vectorized.drop('UniProt', axis=1)
        return (peps, prots, order, agg_prots_vectorized)


class TSHandler(TransformerMixin):
    """A class for transforming set of vectors 
    into 1D vector
    """
    def __init__(self, chosen_idx, column_id, column_sort=None,
                 default_fc_parameters=None, config_function=None):
        self.chosen_idx = chosen_idx
        self.column_id = column_id
        self.column_sort = column_sort
        self.default_fc_parameters = default_fc_parameters
        self.config_function = config_function

    def fit(self, X, y):
        if self.config_function:
            self.default_fc_parameters = \
                self.config_function(X[self.chosen_idx].columns)

        return self

    def transform(self, X):
        agg_data = extract_features(X[self.chosen_idx],
                                    column_id=self.column_id,
                                    column_sort=self.column_sort,
                                    kind_to_fc_parameters=self
                                    .default_fc_parameters)
        agg_data.index.name = self.column_id
        agg_data = agg_data.reset_index()

        # replacing '__' with '_' to avoid problems further
        agg_data.columns = [col.replace('__', '_')
                            for col in agg_data.columns]
        final_list = list(X)
        final_list[self.chosen_idx] = agg_data
        final_tuple = tuple(final_list)
        return final_tuple


class PepInProtHandler(TransformerMixin):
    """A class for transforming set of vectors 
    (which correspond to set of peptides 
    for a certain protein belonging to 
    certain visit) into 1D vector
    """
    def __init__(self, default_fc_parameters={'mean': None},
                 config_function=None):
        self.default_fc_parameters = default_fc_parameters
        self.config_function = config_function

        # We don't want to compute stats for following features
        # since they remain constant for visit-prot combinations
        self.out_of_agg_cols = ['visit_id', 'visit_month',
                                'patient_id', 'UniProt',
                                'NPX', 'Peptide',
                                'PeptideAbundance']

    def _get_pep_prot_combos(self, prot_data, pep_data):
        pep_prot = prot_data \
            .merge(pep_data, on=['visit_id', 'UniProt',
                                 'visit_month', 'patient_id'])[['visit_id',
                                                                'visit_month',
                                                                'patient_id',
                                                                'UniProt',
                                                                'NPX',
                                                                'Peptide']]

        pep_prot_combos = pep_prot \
            .groupby(['visit_id', 'UniProt'])[['UniProt', 'Peptide']] \
            .apply(lambda x: ' ' + '.'.join(x['Peptide']))
        pep_prot_combos = pep_prot_combos.reset_index()
        pep_prot_combos.columns = ['visit_id', 'UniProt', 'pep_prot_combo']
        return pep_prot_combos

    def _remove_duplicate_from_prot_data(self, prot_data, pep_prot_combos):
        pep_prot_combos_rem_dup = pep_prot_combos \
            .drop_duplicates(subset='pep_prot_combo')
        prot_data_rem_dup = prot_data.merge(pep_prot_combos_rem_dup,
                                            on=['visit_id', 'UniProt'])
        return prot_data_rem_dup

    def fit(self, X, y):
        if self.config_function:
            self.default_fc_parameters = self.config_function(X[-1].columns)

        return self

    def transform(self, X):

        peps, prots, order, agg_peps = X

        # Here we remove duplicated combinations of prots and peps
        # for decreasing computational time
        pep_prot_combos = self._get_pep_prot_combos(prots, peps)
        prots_rem_dup = self._remove_duplicate_from_prot_data(prots,
                                                              pep_prot_combos)

        pep_prot_rem_dup = prots_rem_dup \
            .merge(peps, on=['visit_id', 'UniProt',
                             'visit_month', 'patient_id'])
        pep_prot_rem_dup = pep_prot_rem_dup \
            .merge(agg_peps, on=['Peptide']) \
            .drop(self.out_of_agg_cols, axis=1)

        # feature extraction for peptides sets
        agg_agg_peps = extract_features(pep_prot_rem_dup,
                                        column_id='pep_prot_combo',
                                        kind_to_fc_parameters=self
                                        .default_fc_parameters)
        agg_agg_peps.index.name = 'pep_prot_combo'
        agg_agg_peps = agg_agg_peps.reset_index()

        # replacing '__' with '_' to avoid problems further
        agg_agg_peps.columns = [col.replace('__', '_')
                                for col in agg_agg_peps.columns]

        # feature extraction for PeptideAbundance
        agg_abundance = peps.groupby(['visit_id', 'UniProt']) \
            .apply(lambda x: x.PeptideAbundance.mean())
        agg_abundance = agg_abundance.reset_index()
        agg_abundance.columns = ['visit_id', 'UniProt', 'PeptideAbundance']

        # merge everything
        agg_pep_prots = prots \
            .merge(pep_prot_combos, on=['visit_id', 'UniProt']) \
            .merge(agg_agg_peps, on='pep_prot_combo')
        agg_pep_prots = agg_pep_prots.merge(agg_abundance,
                                            on=['visit_id', 'UniProt'])
        agg_pep_prots = agg_pep_prots.drop('pep_prot_combo', axis=1)
        return (peps, prots, order, agg_pep_prots)


class OrderMaintainer(TransformerMixin):
    """A class for maintaining 
    initial order of visits
    """
    def __init__(self):
        pass

    def fit(self, X, y):
        return self

    def transform(self, X):
        peps, prots, order, agg_df = X
        agg_df_ordered = order.merge(agg_df,
                                     on=list(order.columns))
        return (peps, prots, order, agg_df_ordered)


class PatDateExtractor(TransformerMixin):
    """A class for extracting patient_id 
    and visit_month from visit_id
    """
    def __init__(self):
        pass

    def fit(self, X, y):
        return self

    def transform(self, X):
        peps, prots, order, agg_df = X
        agg_df_extracted = agg_df.copy()
        agg_df_extracted['patient_id'] = agg_df_extracted['visit_id'] \
            .apply(lambda x: x.split('_')[0]).astype('int')
        agg_df_extracted['visit_month'] = agg_df_extracted['visit_id'] \
            .apply(lambda x: x.split('_')[1]).astype('int')
        return (peps, prots, order, agg_df_extracted)


class ColumnDropper(TransformerMixin):
    """A class for dropping useless 
    or leak features
    """
    def __init__(self, cols):
        self.cols = cols

    def fit(self, X, y):
        return self

    def transform(self, X):
        return X.drop(self.cols, axis=1)


class DFSelector(TransformerMixin):
    """A class for selecting a certain 
    dataframe from a tuple of 
    several dataframes
    """
    def __init__(self, chosen_idx):
        self.chosen_idx = chosen_idx

    def fit(self, X, y):
        return self

    def transform(self, X):
        return X[self.chosen_idx]


class StandardScalerWrapper(StandardScaler):
    """A wrapper for StandardScaler 
    returning pd.DataFrame
    """
    def __init__(self, *, copy=True,
                 with_mean=True, with_std=True):
        super().__init__(copy=copy, with_mean=with_mean,
                         with_std=with_std)

    def transform(self, X):
        X_scaled = X.copy()
        X_scaled[X_scaled.columns] = super().transform(X)
        X_scaled['visit_id'] = X.visit_id
        return X_scaled


class FeatureSelectorWrapper(TransformerMixin):
    """A convenient wrapper for SelectFromModel 
    """
    def __init__(self, model, target_name,
                 params={}, max_rate_feats=.5):
        self.params = params
        self.model = model
        self.target_name = target_name
        self.max_rate_feats = max_rate_feats

    def _compute_max_features(self, X):
        max_features = int(np.round(X.shape[1] * self.max_rate_feats))

        if max_features == 0:
            max_features = 1

        return max_features

    def fit(self, X, y):
        Xy = X.merge(y[['visit_id', self.target_name]], on='visit_id')
        Xy = Xy.dropna(subset=self.target_name)
        X_ordered = Xy.drop([self.target_name], axis=1)
        y_ordered = Xy[self.target_name]
        max_features = self._compute_max_features(X_ordered)
        self.selector = SelectFromModel(self.model(**self.params),
                                        threshold=-np.inf,
                                        max_features=max_features)
        self.selector.fit(X_ordered, y_ordered)
        return self

    def transform(self, X):
        arr = self.selector.transform(X).astype('float')
        cols = self.selector.get_feature_names_out()
        df = pd.DataFrame(arr, columns=cols)
        df['visit_id'] = X['visit_id']
        return df
    
class ModelWrapper(TransformerMixin):
    """A convenient wrapper for final machine learning model 
    which drops instances where target is unknown 
    """
    def __init__(self, model, target_name, params={}):
        self.model = model(**params)
        self.target_name = target_name

    def fit(self, X, y):
        Xy = X.merge(y[['visit_id', self.target_name]], on='visit_id')
        Xy = Xy.dropna(subset=self.target_name)
        X_ordered = Xy.drop(['visit_id', self.target_name], axis=1)
        y_ordered = Xy[self.target_name]
        self.model.fit(X_ordered, y_ordered)
        return self

    def predict(self, X):
        return self.model.predict(X.drop('visit_id', axis=1))

In [14]:
def make_pipeline(pep_vector_size, pep_window_size,
                  prot_vector_size, prot_window_size,
                  model_for_fs, rate_for_fs, 
                  max_depth, learning_rate, 
                  n_estimators, min_child_weight,
                  gamma, current_target):
    
    param_xgb = {
        'max_depth': max_depth,
        'learning_rate': learning_rate,
        'n_estimators': n_estimators,
        'min_child_weight': min_child_weight,
        'gamma': gamma}

    pipeline = Pipeline([('w2pepvw', W2vPepWrapper(pep_vector_size,
                                                   pep_window_size)),
                         ('tsh1', TSHandler
                          (3, 'Peptide', 'Position',
                           config_function=create_config_pep)),
                         ('ph', PepInProtHandler
                          (config_function=create_config_pep_prot)),
                         ('om1', OrderMaintainer()),
                         ('w2protvw', W2vProtWrapper(prot_vector_size,
                                                     prot_window_size)),
                         ('tsh2', TSHandler
                          (3, 'visit_id', None,
                           config_function=create_config_prot)),
                         ('pde', PatDateExtractor()),
                         ('om2', OrderMaintainer()),
                         ('dfs', DFSelector(-1)),
                         ('cd', ColumnDropper(COL_TO_DEL)),
                         ('sc', StandardScalerWrapper()),
                         ('fsel', FeatureSelectorWrapper
                          (MODELS[model_for_fs],
                           current_target,
                           max_rate_feats=rate_for_fs)),
                         ('model', ModelWrapper(XGBRegressor,
                                                current_target,
                                                params=param_xgb))
                         ])

    return pipeline


def objective(trial):

    global current_target, train_peptides, train_proteins, train_clinical

    pep_vector_size = trial.suggest_int("pep_vector_size", 5, 50)
    pep_window_size = trial.suggest_int("pep_window_size", 3, 20)

    prot_vector_size = trial.suggest_int("prot_vector_size", 5, 50)
    prot_window_size = trial.suggest_int("prot_window_size", 3, 10)

    model_for_fs = trial.suggest_categorical('model_for_fs',
                                             ['ExtraTreeRegressor', 'Ridge'])
    rate_for_fs = trial.suggest_float('rate_for_fs', 0.01, 0.99)

    max_depth = trial.suggest_int('max_depth', 1, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 1.0)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    gamma = trial.suggest_float('gamma', 0.01, 1.0)

    pipe = make_pipeline(pep_vector_size, pep_window_size,
                         prot_vector_size, prot_window_size,
                         model_for_fs, rate_for_fs, 
                         max_depth, learning_rate, 
                         n_estimators, min_child_weight,
                         gamma, current_target)

    cv = KFold(3, shuffle=True, random_state=22)
    patient_ids = train_peptides.patient_id.unique()
    smape_scores = []

    for train_idxs, val_idxs in cv.split(patient_ids):

        smape_score = _fit_and_score_cv(pipe, train_peptides,
                                        train_proteins,
                                        train_clinical,
                                        current_target,
                                        train_idxs, val_idxs)
        smape_scores.append(smape_score)

    mean_smape_score = np.mean(smape_scores)
    return mean_smape_score


def _fit_and_score_cv(pipe, peps, prots,
                      clinical, current_target,
                      train_idxs, val_idxs):
    patient_ids = peps.patient_id.unique()
    train_ids = patient_ids[train_idxs]
    val_ids = patient_ids[val_idxs]

    train_peps, val_peps, \
        train_prots, val_prots, \
        train_clinical, val_clinical = \
        split_cv_data(peps, prots,
                      clinical, train_ids, val_ids)

    train_order = train_peps[[
        'patient_id', 'visit_month']].drop_duplicates()
    val_order = val_peps[[
        'patient_id', 'visit_month']].drop_duplicates()

    pipe.fit((train_peps,
              train_prots,
              train_order), train_clinical)
    preds = pipe.predict((val_peps,
                          val_prots,
                          val_order))

    preds_val_df = val_order.copy()
    preds_val_df['pred'] = preds

    true_pred = preds_val_df \
        .merge(val_clinical, on=['patient_id',
                                 'visit_month'])
    true_pred = true_pred.dropna(subset=current_target)

    smape_score = smape_func(true_pred[current_target].values,
                             true_pred['pred'].values)
    return smape_score

## Brief description of solution

As we have seen during EDA, each patient has several visits. Each visit has a certain set of proteins, and then each protein related to this visit has its own set of peptides, which are represented as sequences of characters. In order to preserve all the information, we've done folowing steps:

1. Applying word2vec for encoding each character of peptides
2. Applying tsfresh for transforming sequence of obtained vectors into 1D vector
3. Applying tsfresh for transforming set of vectors (which correspond to set of peptides for a certain protein belonging to certain visit) into 1D vector
4. Applying word2vec for encoding the names of proteins
5. Applying tsfresh for transforming visit data (encoded protein names belonging to certain visit + data obtained into step 3) into 1D vector
6. Applying xgboost regressor

For tuning hyperparameters of xgboost and each step of preprocessing we've used optuna.

## Reading input data

In [15]:
peptides = pd.read_csv('../data/raw/train_peptides.csv')
proteins = pd.read_csv('../data/raw/train_proteins.csv')
clinical = pd.read_csv('../data/raw/train_clinical_data.csv')

We split all datasets by patients

In [16]:
train_peptides, test_peptides, \
    train_proteins, test_proteins, \
    train_clinical, test_clinical = split_train_val_data(peptides,
                                                         proteins,
                                                         clinical,
                                                         test_size=.2,
                                                         random_state=20)

print(train_clinical.shape, test_clinical.shape)

(2110, 8) (505, 8)


In [17]:
train_order = train_proteins[[
    'patient_id', 'visit_month']].drop_duplicates()
test_order = test_proteins[[
    'patient_id', 'visit_month']].drop_duplicates()

## updrs_1

In [36]:
current_target = 'updrs_1'
study_1 = optuna.create_study()
study_1.optimize(objective, n_trials=10)

[I 2023-04-07 22:00:44,852] A new study created in memory with name: no-name-f1f11a3f-3893-4cea-90a1-e8141942afd0
Feature Extraction: 100%|██████████████████████| 20/20 [00:22<00:00,  1.12s/it]
[I 2023-04-07 22:23:11,767] Trial 0 finished with value: 70.52183713709434 and parameters: {'pep_vector_size': 40, 'pep_window_size': 3, 'prot_vector_size': 43, 'prot_window_size': 9, 'model_for_fs': 'ExtraTreeRegressor', 'threshold_for_ts': 0.6022248267569663, 'max_depth': 7, 'learning_rate': 0.17014878762466404, 'n_estimators': 683, 'min_child_weight': 3, 'gamma': 0.25343698536527814}. Best is trial 0 with value: 70.52183713709434.
Feature Extraction: 100%|██████████████████████| 20/20 [00:14<00:00,  1.34it/s]
[I 2023-04-07 22:38:07,300] Trial 1 finished with value: 74.90081655007675 and parameters: {'pep_vector_size': 26, 'pep_window_size': 18, 'prot_vector_size': 8, 'prot_window_size': 7, 'model_for_fs': 'ExtraTreeRegressor', 'threshold_for_ts': 0.2052980253227712, 'max_depth': 5, 'learning_

Feature Extraction: 100%|██████████████████████| 20/20 [00:47<00:00,  2.36s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-conditioned matrix (rcond=1.55087e-18): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)
Feature Extraction: 100%|██████████████████████| 20/20 [00:45<00:00,  2.28s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-conditioned matrix (rcond=3.55863e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)
Feature Extraction: 100%|██████████████████████| 20/20 [00:25<00:00,  1.26s/it]
[I 2023-04-07 23:23:57,987] Trial 4 finished with value: 87.21809185738056 and parameters: {'pep_vector_size': 45, 'pep_window_size': 5, 'prot_vector_size': 49, 'prot_window_size': 10, 'model_for_fs': 'Ridge', 'threshold_for_ts': 0.42084583117253405, 'max_depth': 9, 'learning_rate': 0.89322403702534

Feature Extraction: 100%|██████████████████████| 20/20 [00:24<00:00,  1.22s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-conditioned matrix (rcond=3.12603e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)
Feature Extraction: 100%|██████████████████████| 20/20 [00:13<00:00,  1.44it/s]
[I 2023-04-07 23:57:14,718] Trial 7 finished with value: 86.59417808814959 and parameters: {'pep_vector_size': 23, 'pep_window_size': 17, 'prot_vector_size': 15, 'prot_window_size': 8, 'model_for_fs': 'Ridge', 'threshold_for_ts': 0.33001245764394616, 'max_depth': 1, 'learning_rate': 0.7122780668511944, 'n_estimators': 310, 'min_child_weight': 8, 'gamma': 0.3859204970681996}. Best is trial 2 with value: 69.83547472232921.
Feature Extraction: 100%|██████████████████████| 20/20 [00:49<00:00,  2.45s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-condition

In [23]:
study_1.best_params

{'pep_vector_size': 17,
 'pep_window_size': 12,
 'prot_vector_size': 23,
 'prot_window_size': 9,
 'model_for_fs': 'ExtraTreeRegressor',
 'rate_for_ts': 0.5118759419179546,
 'max_depth': 6,
 'learning_rate': 0.13158943682751476,
 'n_estimators': 115,
 'min_child_weight': 7,
 'gamma': 0.3083995300599445}

In [41]:
pipe_1 = make_pipeline(current_target=current_target, **study_1.best_params)
pipe_1.fit((train_peptides, train_proteins, train_order), train_clinical)

Feature Extraction: 100%|██████████████████████| 20/20 [00:28<00:00,  1.41s/it]


In [46]:
preds_updrs_1_test = pipe_1.predict((test_peptides, test_proteins, test_order))

Feature Extraction: 100%|██████████████████████| 20/20 [00:09<00:00,  2.13it/s]


In [25]:
preds_test_df = test_order.copy()
preds_test_df['pred'] = preds_updrs_1_test

In [26]:
true_pred = preds_test_df.merge(test_clinical, on = ['patient_id', 'visit_month'])
true_pred = true_pred.dropna(subset='updrs_1')

In [1]:
print(smape_func(true_pred.updrs_1.values, 
                 true_pred['pred'].values))

79.79190020509107

## updrs_2

In [51]:
current_target = 'updrs_2'
study_2 = optuna.create_study()
study_2.optimize(objective, n_trials=10)

[I 2023-04-08 00:57:52,761] A new study created in memory with name: no-name-0f44e85d-dcb0-423f-9e82-62306e4b0200
Feature Extraction: 100%|██████████████████████| 20/20 [00:43<00:00,  2.16s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-conditioned matrix (rcond=1.84858e-18): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)
Feature Extraction: 100%|██████████████████████| 20/20 [00:42<00:00,  2.15s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-conditioned matrix (rcond=1.2622e-18): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)
Feature Extraction: 100%|██████████████████████| 20/20 [00:41<00:00,  2.05s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-conditioned matrix (rcond=3.39923e-17): result may not be accurate.
  dual_coef = linalg.so

Feature Extraction: 100%|██████████████████████| 20/20 [00:21<00:00,  1.10s/it]
[I 2023-04-08 02:18:33,537] Trial 3 finished with value: 115.68998889885198 and parameters: {'pep_vector_size': 37, 'pep_window_size': 9, 'prot_vector_size': 41, 'prot_window_size': 9, 'model_for_fs': 'ExtraTreeRegressor', 'rate_for_fs': 0.3825044651863448, 'max_depth': 2, 'learning_rate': 0.729191609383936, 'n_estimators': 757, 'min_child_weight': 1, 'gamma': 0.013560025242294736}. Best is trial 1 with value: 104.9122780367379.
Feature Extraction: 100%|██████████████████████| 20/20 [00:26<00:00,  1.35s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-conditioned matrix (rcond=1.14811e-18): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)
Feature Extraction: 100%|██████████████████████| 20/20 [00:25<00:00,  1.29s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-c

Feature Extraction: 100%|██████████████████████| 20/20 [00:24<00:00,  1.25s/it]
[I 2023-04-08 03:32:29,438] Trial 7 finished with value: 113.91458677445286 and parameters: {'pep_vector_size': 46, 'pep_window_size': 14, 'prot_vector_size': 41, 'prot_window_size': 4, 'model_for_fs': 'ExtraTreeRegressor', 'rate_for_fs': 0.5660516375609467, 'max_depth': 2, 'learning_rate': 0.8084970455183322, 'n_estimators': 552, 'min_child_weight': 10, 'gamma': 0.8898390420196384}. Best is trial 6 with value: 100.33735059191487.
Feature Extraction: 100%|██████████████████████| 20/20 [00:47<00:00,  2.40s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill-conditioned matrix (rcond=7.81827e-19): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)
Feature Extraction: 100%|██████████████████████| 20/20 [00:47<00:00,  2.38s/it]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:194: LinAlgWarning: Ill

In [54]:
study_2.best_params

{'pep_vector_size': 40,
 'pep_window_size': 8,
 'prot_vector_size': 36,
 'prot_window_size': 5,
 'model_for_fs': 'ExtraTreeRegressor',
 'rate_for_fs': 0.4135469687798218,
 'max_depth': 8,
 'learning_rate': 0.06184973515438345,
 'n_estimators': 263,
 'min_child_weight': 4,
 'gamma': 0.32609672246286237}

In [23]:
pipe_2 = make_pipeline(current_target=current_target, **study_2.best_params)
pipe_2.fit((train_peptides, train_proteins, train_order), train_clinical)

Feature Extraction: 100%|██████████| 20/20 [01:31<00:00,  4.58s/it]


In [28]:
preds_updrs_2_test = pipe_2.predict((test_peptides, test_proteins, test_order))

Feature Extraction: 100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
/home/max/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/home/max/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [29]:
preds_test_df = test_order.copy()
preds_test_df['pred'] = preds_updrs_2_test

In [30]:
true_pred = preds_test_df.merge(test_clinical, on = ['patient_id', 'visit_month'])
true_pred = true_pred.dropna(subset='updrs_2')

In [2]:
print(smape_func(true_pred.updrs_2.values, 
                 true_pred['pred'].values))

104.39986880561862

## updrs_3

In [52]:
current_target = 'updrs_3'
study_3 = optuna.create_study()
study_3.optimize(objective, n_trials=10)

[I 2023-04-08 04:22:22,397] A new study created in memory with name: no-name-099a833f-535a-4b62-9819-d3f817069d2c
Feature Extraction: 100%|██████████████████████| 20/20 [00:13<00:00,  1.49it/s]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=9.41173e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
Feature Extraction: 100%|██████████████████████| 20/20 [00:13<00:00,  1.50it/s]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.86734e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
Feature Extraction: 100%|██████████████████████| 20/20 [00:08<00:00,  2.41it/s]
[I 2023-04-08 04:30:40,764] Trial 0 finished with value: 114.60439299406926 and parameters: {'pep_vector_size': 11, 'pep_window_size': 16, 'prot_vector_size': 14, 'prot_window_size': 8, 'model

Feature Extraction: 100%|██████████████████████| 20/20 [00:24<00:00,  1.21s/it]
[I 2023-04-08 05:31:09,377] Trial 3 finished with value: 99.28795747868656 and parameters: {'pep_vector_size': 45, 'pep_window_size': 9, 'prot_vector_size': 37, 'prot_window_size': 4, 'model_for_fs': 'Ridge', 'rate_for_fs': 0.1367662510908947, 'max_depth': 2, 'learning_rate': 0.27533119767858366, 'n_estimators': 933, 'min_child_weight': 3, 'gamma': 0.27867934316679777}. Best is trial 3 with value: 99.28795747868656.
Feature Extraction: 100%|██████████████████████| 20/20 [00:25<00:00,  1.26s/it]
[I 2023-04-08 05:57:20,498] Trial 4 finished with value: 92.31042418115045 and parameters: {'pep_vector_size': 49, 'pep_window_size': 13, 'prot_vector_size': 8, 'prot_window_size': 10, 'model_for_fs': 'ExtraTreeRegressor', 'rate_for_fs': 0.2918732060340875, 'max_depth': 7, 'learning_rate': 0.02591584266122024, 'n_estimators': 634, 'min_child_weight': 2, 'gamma': 0.5164584865786107}. Best is trial 4 with value: 92.310

Feature Extraction: 100%|██████████████████████| 20/20 [00:19<00:00,  1.02it/s]
[I 2023-04-08 06:43:52,194] Trial 7 finished with value: 111.70527067630753 and parameters: {'pep_vector_size': 35, 'pep_window_size': 6, 'prot_vector_size': 40, 'prot_window_size': 7, 'model_for_fs': 'ExtraTreeRegressor', 'rate_for_fs': 0.4844117212810542, 'max_depth': 7, 'learning_rate': 0.9554484164471532, 'n_estimators': 547, 'min_child_weight': 1, 'gamma': 0.1321276332748439}. Best is trial 4 with value: 92.31042418115045.
Feature Extraction: 100%|██████████████████████| 20/20 [00:17<00:00,  1.12it/s]
[I 2023-04-08 07:01:38,667] Trial 8 finished with value: 97.31912892272639 and parameters: {'pep_vector_size': 32, 'pep_window_size': 19, 'prot_vector_size': 19, 'prot_window_size': 7, 'model_for_fs': 'ExtraTreeRegressor', 'rate_for_fs': 0.21459566144687262, 'max_depth': 8, 'learning_rate': 0.33686814341884963, 'n_estimators': 697, 'min_child_weight': 5, 'gamma': 0.06514508759624503}. Best is trial 4 with

In [58]:
study_3.best_params

{'pep_vector_size': 49,
 'pep_window_size': 13,
 'prot_vector_size': 8,
 'prot_window_size': 10,
 'model_for_fs': 'ExtraTreeRegressor',
 'rate_for_fs': 0.2918732060340875,
 'max_depth': 7,
 'learning_rate': 0.02591584266122024,
 'n_estimators': 634,
 'min_child_weight': 2,
 'gamma': 0.5164584865786107}

In [36]:
pipe_3 = make_pipeline(current_target=current_target, **study_3.best_params)
pipe_3.fit((train_peptides, train_proteins, train_order), train_clinical)

Feature Extraction:  65%|██████▌   | 13/20 [03:49<01:59, 17.10s/it]

: 

: 

In [26]:
preds_updrs_3_test = pipe_3.predict((test_peptides, test_proteins, test_order))

Feature Extraction: 100%|██████████| 20/20 [00:19<00:00,  1.00it/s]


In [27]:
preds_test_df = test_order.copy()
preds_test_df['pred'] = preds_updrs_3_test

In [28]:
true_pred = preds_test_df.merge(test_clinical, on = ['patient_id', 'visit_month'])
true_pred = true_pred.dropna(subset='updrs_3')

In [3]:
print(smape_func(true_pred.updrs_3.values, 
                 true_pred['pred'].values))

119.47978284515281

## updrs_4

In [53]:
current_target = 'updrs_4'
study_4 = optuna.create_study()
study_4.optimize(objective, n_trials=10)

[I 2023-04-08 07:11:06,274] A new study created in memory with name: no-name-b35a5e52-c023-46ad-a53f-c16ba7456c74
Feature Extraction: 100%|██████████████████████| 20/20 [00:10<00:00,  1.85it/s]
C:\Users\Max\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
C:\Users\Max\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
[I 2023-04-08 07:21:13,496] Trial 0 finished with value: 154.38223689918618 and parameters: {'pep_vector_size': 15, 'pep_window_size': 15, 'prot_vector_size': 45, 'prot_window_size': 5, 'model_for_fs': 'ExtraTreeRegressor'

Feature Extraction: 100%|██████████████████████| 20/20 [00:17<00:00,  1.14it/s]
[I 2023-04-08 08:11:01,085] Trial 3 finished with value: 156.6150913734269 and parameters: {'pep_vector_size': 30, 'pep_window_size': 6, 'prot_vector_size': 36, 'prot_window_size': 5, 'model_for_fs': 'ExtraTreeRegressor', 'rate_for_fs': 0.8955231943074614, 'max_depth': 8, 'learning_rate': 0.1850852096679211, 'n_estimators': 788, 'min_child_weight': 7, 'gamma': 0.17578742871350161}. Best is trial 0 with value: 154.38223689918618.
Feature Extraction: 100%|██████████████████████| 20/20 [00:23<00:00,  1.18s/it]
[I 2023-04-08 08:34:38,855] Trial 4 finished with value: 159.95564421058114 and parameters: {'pep_vector_size': 44, 'pep_window_size': 20, 'prot_vector_size': 31, 'prot_window_size': 9, 'model_for_fs': 'ExtraTreeRegressor', 'rate_for_fs': 0.49765480441926335, 'max_depth': 2, 'learning_rate': 0.41294260585710574, 'n_estimators': 168, 'min_child_weight': 3, 'gamma': 0.7853882749955646}. Best is trial 0 wit

Feature Extraction: 100%|██████████████████████| 20/20 [00:22<00:00,  1.14s/it]
[I 2023-04-08 09:31:20,874] Trial 7 finished with value: 158.5517529344165 and parameters: {'pep_vector_size': 43, 'pep_window_size': 8, 'prot_vector_size': 16, 'prot_window_size': 10, 'model_for_fs': 'ExtraTreeRegressor', 'rate_for_fs': 0.7646917714718511, 'max_depth': 2, 'learning_rate': 0.28873668089780236, 'n_estimators': 517, 'min_child_weight': 3, 'gamma': 0.509656620269127}. Best is trial 0 with value: 154.38223689918618.
Feature Extraction: 100%|██████████████████████| 20/20 [00:12<00:00,  1.66it/s]
[I 2023-04-08 09:43:22,685] Trial 8 finished with value: 160.12161363558616 and parameters: {'pep_vector_size': 20, 'pep_window_size': 6, 'prot_vector_size': 10, 'prot_window_size': 4, 'model_for_fs': 'ExtraTreeRegressor', 'rate_for_fs': 0.058232203183018715, 'max_depth': 10, 'learning_rate': 0.13229269035623864, 'n_estimators': 940, 'min_child_weight': 2, 'gamma': 0.4556307133342436}. Best is trial 0 wi

In [59]:
study_4.best_params

{'pep_vector_size': 15,
 'pep_window_size': 15,
 'prot_vector_size': 45,
 'prot_window_size': 5,
 'model_for_fs': 'ExtraTreeRegressor',
 'rate_for_fs': 0.14939865620600673,
 'max_depth': 1,
 'learning_rate': 0.5531303746999062,
 'n_estimators': 124,
 'min_child_weight': 6,
 'gamma': 0.7366981712692301}

In [27]:
pipe_4 = make_pipeline(current_target=current_target, **study_4.best_params)
pipe_4.fit((train_peptides, train_proteins, train_order), train_clinical)

Feature Extraction: 100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


In [31]:
preds_updrs_4_test = pipe_4.predict((test_peptides, test_proteins, test_order))

Feature Extraction: 100%|██████████| 20/20 [00:07<00:00,  2.55it/s]


In [32]:
preds_test_df = test_order.copy()
preds_test_df['pred'] = preds_updrs_4_test

In [33]:
true_pred = preds_test_df.merge(test_clinical, on = ['patient_id', 'visit_month'])
true_pred = true_pred.dropna(subset='updrs_4')

In [4]:
print(smape_func(true_pred.updrs_4.values, 
                 true_pred['pred'].values))

162.736571157652